# EP1 - Modelo treinado

Felipe
Nilton

## Descricao

In [1]:
#!pip install -r requirements.txt
import pandas as pd
import numpy as np

In [2]:
np.random.seed(777)

In [3]:
df = pd.read_excel('datasets/ep1_esic2023_clareza_TRAIN.xlsx')
df.head()

,resp_text,clarity
0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5
1,"""A pedido do Pró-Reitor de Graduação, informa...",c5
2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234
3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234
4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   resp_text  6000 non-null   object
 1   clarity    6000 non-null   object
dtypes: object(2)
memory usage: 93.9+ KB


In [5]:
df.describe()

,resp_text,clarity
count,6000,6000
unique,5626,3
top,"Prezado(a) Senhor(a), Sua manifestação foi a...",c5
freq,41,2000


In [6]:
df.groupby('clarity').describe()

resp_text                                                            \
            count unique                                                top   
clarity                                                                       
c1           2000   1839   Prezado(a) Senhor(a),  Sua manifestação foi a...   
c234         2000   1929   Prezado(a) Senhor(a),  Sua manifestação foi a...   
c5           2000   1911   Prezado (a) Senhor (a)     1. Em atenção ao p...   

              
        freq  
clarity       
c1        24  
c234      17  
c5         8

## Pre-processing

### Class encode

In [7]:
l_unique = sorted(df['clarity'].unique())
label_map = {x: i for i, x in enumerate(l_unique)} 
label_map

{'c1': 0, 'c234': 1, 'c5': 2}

In [8]:
df['Class'] = df['clarity'].map(label_map)

### I. Cleaning

In [9]:
import string

In [10]:
df.sort_values(by='clarity', ascending=True, inplace=True, ignore_index=True)
df.head()

,resp_text,clarity,Class
0,"Prezada Senhora, Informa-se que o DNIT é uma ...",c1,0
1,"Prezada, Os dados se referem somente aos na...",c1,0
2,"Prezado Usuário, Inicialmente agradecemos o s...",c1,0
3,"Prezada, Referimo-nos a sua demanda, regist...",c1,0
4,"Prezada, Segue anexa a resposta da área com...",c1,0


In [11]:
df['resp_text'].dropna(inplace=True)

In [12]:
texts = df['resp_text']
texts = texts.str.lower()
texts = texts.str.translate(str.maketrans('', '', string.punctuation))
#texts = texts.str.replace('[\d+]', '') #remove numeros

df['tokens'] = texts

### II. Tokenize

In [13]:
from nltk.tokenize import word_tokenize

In [14]:
df['tokens'] = [word_tokenize(resp) for resp in df['tokens']]
df.head(10)

,resp_text,clarity,Class,tokens
0,"Prezada Senhora, Informa-se que o DNIT é uma ...",c1,0,"[prezada, senhora, informase, que, o, dnit, é,..."
1,"Prezada, Os dados se referem somente aos na...",c1,0,"[prezada, os, dados, se, referem, somente, aos..."
2,"Prezado Usuário, Inicialmente agradecemos o s...",c1,0,"[prezado, usuário, inicialmente, agradecemos, ..."
3,"Prezada, Referimo-nos a sua demanda, regist...",c1,0,"[prezada, referimonos, a, sua, demanda, regist..."
4,"Prezada, Segue anexa a resposta da área com...",c1,0,"[prezada, segue, anexa, a, resposta, da, área,..."
5,"Prezada, Segue anexa a resposta da área com...",c1,0,"[prezada, segue, anexa, a, resposta, da, área,..."
6,"Prezada, Segue em anexo resposta da área té...",c1,0,"[prezada, segue, em, anexo, resposta, da, área..."
7,"Prezado usuário, Segue anexa resposta do seto...",c1,0,"[prezado, usuário, segue, anexa, resposta, do,..."
8,"Prezada, Segue resposta ao seu pedido de in...",c1,0,"[prezada, segue, resposta, ao, seu, pedido, de..."
9,"Prezado Usuário, Enviamos a resposta apresent...",c1,0,"[prezado, usuário, enviamos, a, resposta, apre..."


### III. Lemma

In [15]:
import spacy
import os

In [16]:
try:
    nlp = spacy.load("pt_core_news_sm")
except Exception:
    os.system("python -m spacy download pt_core_news_sm")
    nlp = spacy.load("pt_core_news_sm")

In [17]:
filtered = []
for sent in df['tokens']:
    sent = str(sent)
    doc = nlp(sent)
    temp = [token.lemma_ for token in doc if not token.is_punct]
    temp = " ".join(temp)
    filtered.append(temp)

In [18]:
df['lemma'] = filtered
df.head()

,resp_text,clarity,Class,tokens,lemma
0,"Prezada Senhora, Informa-se que o DNIT é uma ...",c1,0,"[prezada, senhora, informase, que, o, dnit, é,...",prezar senhor informase que o dnit ser um auta...
1,"Prezada, Os dados se referem somente aos na...",c1,0,"[prezada, os, dados, se, referem, somente, aos...",prezar o dar se referir somente a o nascido vi...
2,"Prezado Usuário, Inicialmente agradecemos o s...",c1,0,"[prezado, usuário, inicialmente, agradecemos, ...",prezar usuário inicialmente agradecer o seu co...
3,"Prezada, Referimo-nos a sua demanda, regist...",c1,0,"[prezada, referimonos, a, sua, demanda, regist...",prezar referimono o seu demanda registrar em o...
4,"Prezada, Segue anexa a resposta da área com...",c1,0,"[prezada, segue, anexa, a, resposta, da, área,...",prezar seguir anexo o resposta de o área compe...


### IV N-gram

In [19]:
import nltk
from nltk.util import ngrams

In [20]:
for i in range(2,4):
    temp = []

    for txt in df['lemma']:    
        txt = list(ngrams(sequence=nltk.word_tokenize(txt), n=i))
        temp.append(txt)
    
    df['ngram' + str(i)] = temp
    

In [21]:
df.head()

,resp_text,clarity,Class,tokens,lemma,ngram2,ngram3
0,"Prezada Senhora, Informa-se que o DNIT é uma ...",c1,0,"[prezada, senhora, informase, que, o, dnit, é,...",prezar senhor informase que o dnit ser um auta...,"[(prezar, senhor), (senhor, informase), (infor...","[(prezar, senhor, informase), (senhor, informa..."
1,"Prezada, Os dados se referem somente aos na...",c1,0,"[prezada, os, dados, se, referem, somente, aos...",prezar o dar se referir somente a o nascido vi...,"[(prezar, o), (o, dar), (dar, se), (se, referi...","[(prezar, o, dar), (o, dar, se), (dar, se, ref..."
2,"Prezado Usuário, Inicialmente agradecemos o s...",c1,0,"[prezado, usuário, inicialmente, agradecemos, ...",prezar usuário inicialmente agradecer o seu co...,"[(prezar, usuário), (usuário, inicialmente), (...","[(prezar, usuário, inicialmente), (usuário, in..."
3,"Prezada, Referimo-nos a sua demanda, regist...",c1,0,"[prezada, referimonos, a, sua, demanda, regist...",prezar referimono o seu demanda registrar em o...,"[(prezar, referimono), (referimono, o), (o, se...","[(prezar, referimono, o), (referimono, o, seu)..."
4,"Prezada, Segue anexa a resposta da área com...",c1,0,"[prezada, segue, anexa, a, resposta, da, área,...",prezar seguir anexo o resposta de o área compe...,"[(prezar, seguir), (seguir, anexo), (anexo, o)...","[(prezar, seguir, anexo), (seguir, anexo, o), ..."


In [22]:
df.to_csv('./datasets/DfCleaned.csv')